In [1]:
import pandas as pd
import ast
import matplotlib.pyplot as plt
path_toData = r'‪C:\Users\spenc\OneDrive - The University of Auckland\Study\UoA\Project\matching_new.csv'
data = pd.read_csv(path_toData.strip("‪u202a"),header=0)
data.head(n=5)
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource
from bokeh.models.tools import HoverTool

In [2]:
newData = data

In [3]:
len(newData.index)

1098

In [4]:
data.fillna('').astype(str).apply(lambda x:x.str.len()).mean()

text                  763.473842
entities              197.958894
overall_confidence     11.108371
dtype: float64

In [18]:
confidence = ColumnDataSource(data)

In [22]:
hist, edges = np.histogram(data.overall_confidence, density=True, bins=100)

In [60]:
import statistics
from bokeh.models import BoxAnnotation, FixedTicker
p = figure(x_axis_label = "Textract Confidence Value", title = "Histogram of OCR Confidence Score", y_axis_label = '')
p.yaxis.major_label_text_font_size = '0pt'  # turn off x-axis tick labels
p.yaxis.major_label_text_font_size = '0pt'  # turn off y-axis tick labels
p.yaxis.major_tick_line_color = None  # turn off y-axis major ticks
p.yaxis.minor_tick_line_color = None  # turn off y-axis minor ticks
high_box = BoxAnnotation(left=90, right= 100, fill_alpha=0.1, fill_color='green')
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="white")
p.add_layout(high_box)
p.xaxis.ticker = FixedTicker(ticks=[0,20,40,60,80,90,96,100])
p.plot_height = 600
p.plot_width = 1000
show(p)

In [59]:
min(data.overall_confidence)

17.37746906

In [5]:
len(data.index)

1338

In [3]:
seriesLower = newData['text'].apply(lambda x: x.lower())
entities = newData['entities'].apply(lambda x: ast.literal_eval(x))
dataLower = seriesLower.to_frame().join(entities)
dataLower.head(n=5)

,text,entities
0,"gourmet burger kitchen, 45 greenwich church, s...","{'entities': [(275, 283, 'DATE'), (269, 274, '..."
1,"suncorp, bank, quick easy conuenien, brisbane ...","{'entities': [(319, 329, 'DATE'), (330, 335, '..."
2,"c abcharge h, tax invoice, silvertop vic 13100...","{'entities': [(258, 266, 'DATE'), (201, 207, '..."
3,"nz drycleaners, 14 polygon road, st heliers, *...","{'entities': [(90, 95, 'TIME'), (189, 198, 'AM..."
4,"hmshost nz limited, auckland int airport, (09)...","{'entities': [(96, 101, 'TIME'), (199, 208, 'A..."


In [7]:
import spacy 
sp = spacy.load('en_core_web_lg')
dataToken = dataLower['text'].apply(lambda x: sp(x))
dataEntities = dataToken.to_frame().join(dataLower['entities'])
dataEntities.head(n=5)
dataEntities = dataEntities[-dataEntities['text'].apply(lambda x: len(x) > 500)]
def entitySetter(_token, _inputDataSeries):
    if len(_inputDataSeries['entities']) > 1:
        for entity in _inputDataSeries['entities']:
            if _token.idx >= int(entity[0]) and _token.idx < int(entity[1]):
                if _token.idx == int(entity[0]):
                    _token.ent_type_ = "B-" + entity[2]
                else:
                    _token.ent_type_ = "I-" + entity[2]
                break
            else:
                    _token.ent_type_ = 'O'
    else:
        _token.ent_type_ = 'O'
    return _token
dataEntities.iloc[0]
len(dataEntities.index)
i = 0
while i < len(dataEntities.index):
    for token in dataEntities.iloc[i]['text']:
        token = entitySetter(token, dataEntities.iloc[i]['entities'])
    i = i + 1
rows = []
i = 0
while i < len(dataEntities.index):
    j = 0
    while j < len(dataEntities.iloc[i]['text']):
        rows.append(['invoice_' + str(i + 1), dataEntities.iloc[i]['text'][j].text, dataEntities.iloc[i]['text'][j].pos_, dataEntities.iloc[i]['text'][j].ent_type_])
        j = j+1
    i = i +1
preproccessDf = pd.DataFrame(rows, columns = ['invoice_id', 'token', 'POS', 'tag'])
preproccessDf['tag'].unique()
preproccessDf.to_csv('BIO_tagged_data_conf.csv', index = False)

In [4]:
import spacy 
sp = spacy.load('en_core_web_lg')
dataToken = dataLower['text'].apply(lambda x: sp(x))
dataEntities = dataToken.to_frame().join(dataLower['entities'])
dataEntities.head(n=5)
dataEntities = dataEntities[-dataEntities['text'].apply(lambda x: len(x) > 500)]
def entitySetter(_token, _inputDataSeries):
    if len(_inputDataSeries['entities']) > 1:
        for entity in _inputDataSeries['entities']:
            if _token.idx >= int(entity[0]) and _token.idx < int(entity[1]):
                if _token.idx == int(entity[0]):
                    _token.ent_type_ = "B-" + entity[2]
                else:
                    _token.ent_type_ = "I-" + entity[2]
                break
            else:
                    _token.ent_type_ = 'O'
    else:
        _token.ent_type_ = 'O'
    return _token
dataEntities.iloc[0]
len(dataEntities.index)

1281

In [7]:
BIO_tagged_data = pd.read_csv('BIO_tagged_data.csv', header = 0)
BIO_tagged_data = BIO_tagged_data.drop(['POS'], axis =1)
BIO_tagged_data_conf = pd.read_csv('BIO_tagged_data_conf.csv', header = 0)
BIO_tagged_data_conf = BIO_tagged_data_conf.drop(['POS'], axis =1)

In [10]:
BIO_tagged_data = BIO_tagged_data[BIO_tagged_data.tag != 'O']
BIO_tagged_data_conf = BIO_tagged_data_conf[BIO_tagged_data_conf.tag != 'O']

In [15]:
round(BIO_tagged_data_conf['tag'].value_counts(normalize=True)*100,2)

B-AMOUNT      20.80
I-AMOUNT      20.04
I-MERCHANT    11.20
B-MERCHANT    10.18
B-DATE        10.14
I-DATE         9.75
I-GST          7.86
B-TIME         6.25
B-GST          1.95
I-TIME         1.84
Name: tag, dtype: float64